# 🚀 Valentina FLUX.1-Dev LoRA Trainer v2.0 - Google Colab Edition

Este notebook treina uma **LoRA de identidade facial** para a Valentina usando **FLUX.1-dev** com a stack **FluxGym + Kohya ss** no Google Colab com GPU A100.

## 🎯 Objetivo
- **Treinar LoRA de identidade visual** da Valentina com máxima qualidade
- **Dataset**: 18 imagens de alta qualidade em 1024x1024 
- **Trigger Word**: `vltna woman`
- **Resultado**: LoRA funcional para geração de imagens personalizadas

## ⚠️ Requisitos OBRIGATÓRIOS
- **Google Colab Pro/Pro+** (GPU A100 necessária)
- **Dataset de 18 imagens** da Valentina em formato PNG 1024x1024
- **Tempo estimado**: 2-3 horas de treinamento

## 📚 Referências
Baseado no guia completo de treinamento FLUX LoRA com FluxGym e metodologias estabelecidas da comunidade.

---

**🔥 Execute as células em ordem e monitore o progresso!**

## 📋 Checklist Pré-Treinamento

Antes de iniciar, certifique-se de que:

- [ ] **GPU A100 ativada** no Colab Pro/Pro+
- [ ] **18 imagens da Valentina** preparadas (PNG, 1024x1024, diversificadas)
- [ ] **Legendas preparadas** com trigger word `vltna woman`
- [ ] **Conexão estável** para evitar desconexão durante treinamento
- [ ] **Pelo menos 3-4 horas livres** para monitoramento

### 🎯 Configurações do Treinamento
```
Modelo Base: FLUX.1-dev (FP8)
Dataset: 18 imagens 1024x1024
Trigger: vltna woman
Repeat: 10 (180 passos por época)
Epochs: 10 (≈1800 passos totais)
Learning Rate: 1e-4
Rank: 16
Batch Size: 2 (se VRAM permitir)
```

In [ ]:
# 🔧 CONFIGURAÇÕES PRINCIPAIS
import os
import subprocess
import time
from pathlib import Path

# === CONFIGURAÇÕES DO TREINAMENTO ===
TRIGGER_WORD = "vltna woman"  # Palavra-chave exclusiva da Valentina
LORA_NAME = "valentina_flux_identity_lora"  # Nome do arquivo LoRA final
DATASET_SIZE = 18  # Número de imagens no dataset

# === PARÂMETROS OTIMIZADOS PARA IDENTIDADE FACIAL ===
REPEAT_COUNT = 10          # Repetições por imagem por época  
MAX_EPOCHS = 10            # Número de épocas (≈1800 passos totais)
LEARNING_RATE = 1e-4       # Learning rate conservador para identidade
NETWORK_RANK = 16          # Rank do LoRA (16 é ideal para identidade)
BATCH_SIZE = 2             # Batch size (ajustar conforme VRAM)

# === CAMINHOS NO COLAB ===
FLUXGYM_PATH = "/content/fluxgym-Colab"
MODELS_PATH = "/content/fluxgym-Colab/models"
DATASET_PATH = "/content/dataset"
OUTPUT_PATH = "/content/fluxgym-Colab/outputs"

print("🎯 Configurações carregadas!")
print(f"📊 Treinamento: {DATASET_SIZE} imagens × {REPEAT_COUNT} repeats × {MAX_EPOCHS} epochs = {DATASET_SIZE * REPEAT_COUNT * MAX_EPOCHS} passos")
print(f"🎨 Trigger word: '{TRIGGER_WORD}'")
print(f"📁 LoRA final: {LORA_NAME}.safetensors")

## 🚀 Etapa 1: Configuração do Ambiente

Instalação do FluxGym, Kohya ss e todas as dependências necessárias para treinamento FLUX LoRA.

In [ ]:
# 🔄 Verificar GPU disponível
def check_gpu():
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ GPU NVIDIA detectada:")
            print(result.stdout.split('\n')[8:12])  # Mostrar info da GPU
            return True
        else:
            print("❌ GPU NVIDIA não encontrada!")
            return False
    except:
        print("❌ nvidia-smi não disponível!")
        return False

# Verificar GPU antes de continuar
if not check_gpu():
    raise RuntimeError("⚠️ GPU A100 necessária! Ative GPU no Runtime > Change runtime type")

print("\n🎉 GPU verificada! Prosseguindo com a instalação...")

In [ ]:
# 📦 Clonar FluxGym e sd-scripts (Kohya)
print("📥 Clonando FluxGym...")
!git clone https://github.com/TheLocalLab/fluxgym-Colab.git 
%cd /content/fluxgym-Colab/

print("\n📥 Clonando sd-scripts (Kohya) - branch sd3 para FLUX...")
!git clone -b sd3 https://github.com/kohya-ss/sd-scripts

print("\n✅ Repositórios clonados com sucesso!")

In [ ]:
# 🔧 Instalar dependências do sd-scripts
print("📦 Instalando dependências do sd-scripts...")
%cd /content/fluxgym-Colab/sd-scripts
!pip install -r requirements.txt

print("\n📦 Instalando dependências do FluxGym...")
%cd /content/fluxgym-Colab/
!pip install -r requirements.txt

print("\n📦 Instalando PyTorch Nightly com suporte FP8 para CUDA 12.1...")
!pip install --pre torch==2.4.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("\n✅ Todas as dependências instaladas!")

## 📁 Etapa 2: Download dos Modelos FLUX.1-Dev

Download de todos os componentes necessários do modelo FLUX.1-dev (UNet, Text Encoders, VAE).

In [ ]:
# 🗂️ Criar estrutura de pastas para os modelos
!mkdir -p /content/fluxgym-Colab/models/unet
!mkdir -p /content/fluxgym-Colab/models/clip  
!mkdir -p /content/fluxgym-Colab/models/vae

print("✅ Estrutura de pastas criada!")

In [ ]:
# 📥 Download do modelo UNet FLUX-Dev (FP8)
print("📥 Baixando FLUX.1-dev UNet (FP8) - ~17GB...")
!wget -O /content/fluxgym-Colab/models/unet/flux1-dev-fp8.safetensors \
     https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-dev-fp8.safetensors

print("\n✅ UNet FLUX.1-dev baixado!")

In [ ]:
# 📥 Download dos Text Encoders
print("📥 Baixando CLIP-L Text Encoder...")
!wget -O /content/fluxgym-Colab/models/clip/clip_l.safetensors \
     https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors

print("\n📥 Baixando T5-XXL Text Encoder (FP8)...")
!wget -O /content/fluxgym-Colab/models/clip/t5xxl_fp8.safetensors \
     https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors

print("\n✅ Text Encoders baixados!")

In [ ]:
# 📥 Download do VAE
print("📥 Baixando VAE do FLUX...")
!wget -O /content/fluxgym-Colab/models/vae/ae.sft \
     https://huggingface.co/cocktailpeanut/xulf-dev/resolve/main/ae.sft

print("\n✅ VAE baixado!")
print("\n🎉 Todos os componentes do FLUX.1-dev foram baixados com sucesso!")

## 📸 Etapa 3: Preparação do Dataset

Upload e configuração das 18 imagens da Valentina com suas respectivas legendas.

In [ ]:
# 📁 Setup do Dataset com Upload Automático
from google.colab import files
import zipfile
import os
import shutil
from PIL import Image

def setup_dataset_from_zip():
    """
    Faz upload do dataset ZIP e organiza automaticamente as imagens
    """
    print("📦 UPLOAD DO DATASET ZIP")
    print("=" * 50)
    print("🔼 Selecione o arquivo ZIP do dataset da Valentina...")
    print("   Arquivo esperado: valentina_identity_4lora_dataset_flux.zip")
    print("   Contendo: 18 imagens PNG em 1024x1024")
    
    # Upload do arquivo ZIP
    uploaded = files.upload()
    
    if not uploaded:
        raise ValueError("❌ Nenhum arquivo foi enviado!")
    
    # Pegar o primeiro arquivo enviado
    zip_filename = list(uploaded.keys())[0]
    print(f"\n📥 Arquivo recebido: {zip_filename}")
    
    # Verificar se é um arquivo ZIP
    if not zip_filename.lower().endswith('.zip'):
        raise ValueError("❌ Arquivo deve ser um ZIP!")
    
    return zip_filename

def extract_and_organize_dataset(zip_filename):
    """
    Extrai o ZIP e organiza as imagens no dataset
    """
    print(f"\n📂 Extraindo {zip_filename}...")
    
    # Criar pasta temporária para extração
    temp_dir = "/content/temp_extract"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Extrair ZIP
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
    
    # Criar pasta final do dataset
    os.makedirs(DATASET_PATH, exist_ok=True)
    
    # Encontrar todas as imagens no ZIP extraído
    image_files = []
    for root, dirs, files in os.walk(temp_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
    
    # Ordenar arquivos
    image_files.sort()
    
    print(f"\n🖼️ Encontradas {len(image_files)} imagens no ZIP")
    
    # Copiar e renomear imagens sequencialmente
    copied_files = []
    for i, img_path in enumerate(image_files, 1):
        # Gerar nome sequencial
        new_filename = f"valentina_{i:02d}.png"
        new_path = os.path.join(DATASET_PATH, new_filename)
        
        # Copiar arquivo
        shutil.copy2(img_path, new_path)
        copied_files.append(new_filename)
        
        # Verificar dimensões da imagem
        try:
            with Image.open(new_path) as img:
                print(f"   ✅ {new_filename} - {img.size[0]}x{img.size[1]} - {img.format}")
        except Exception as e:
            print(f"   ❌ {new_filename} - ERRO: {e}")
    
    # Limpar pasta temporária
    shutil.rmtree(temp_dir)
    os.remove(zip_filename)  # Remover ZIP original
    
    return copied_files

def validate_dataset(image_files):
    """
    Valida o dataset extraído
    """
    global DATASET_SIZE  # Declaração global no início da função
    
    print(f"\n🔍 VALIDAÇÃO DO DATASET")
    print("=" * 30)
    
    valid_count = 0
    invalid_files = []
    
    for filename in image_files:
        file_path = os.path.join(DATASET_PATH, filename)
        
        try:
            with Image.open(file_path) as img:
                # Verificar se é 1024x1024
                if img.size == (1024, 1024):
                    valid_count += 1
                else:
                    invalid_files.append(f"{filename} - Resolução {img.size[0]}x{img.size[1]} (esperado 1024x1024)")
        except Exception as e:
            invalid_files.append(f"{filename} - Erro: {e}")
    
    # Relatório de validação
    print(f"✅ Imagens válidas: {valid_count}/{len(image_files)}")
    
    if invalid_files:
        print(f"⚠️ Problemas encontrados:")
        for issue in invalid_files:
            print(f"   • {issue}")
    
    # Verificar se temos o número esperado
    if len(image_files) != DATASET_SIZE:
        print(f"\n⚠️ ATENÇÃO: Esperado {DATASET_SIZE} imagens, encontrado {len(image_files)}")
        
        if len(image_files) < DATASET_SIZE:
            print("   📊 Ajustando configurações para o número de imagens disponível...")
            DATASET_SIZE = len(image_files)  # Agora pode modificar a variável global
            print(f"   🔄 Novo tamanho do dataset: {DATASET_SIZE} imagens")
    
    return valid_count == len(image_files)

# Executar setup completo do dataset
try:
    print("🚀 CONFIGURAÇÃO AUTOMÁTICA DO DATASET")
    print("=" * 50)
    
    # 1. Upload do ZIP
    zip_file = setup_dataset_from_zip()
    
    # 2. Extrair e organizar
    dataset_files = extract_and_organize_dataset(zip_file)
    
    # 3. Validar dataset
    is_valid = validate_dataset(dataset_files)
    
    # 4. Resultado final
    print(f"\n🎉 DATASET CONFIGURADO COM SUCESSO!")
    print(f"📁 Pasta: {DATASET_PATH}")
    print(f"📊 Imagens: {len(dataset_files)}")
    print(f"🎯 Status: {'✅ Válido' if is_valid else '⚠️ Com problemas'}")
    
    if is_valid:
        print(f"\n✅ Pronto para gerar legendas e iniciar o treinamento!")
        print(f"📊 Configuração atualizada: {DATASET_SIZE} imagens × {REPEAT_COUNT} repeats × {MAX_EPOCHS} epochs = {DATASET_SIZE * REPEAT_COUNT * MAX_EPOCHS} passos")
    else:
        print(f"\n⚠️ Verifique os problemas reportados acima antes de continuar.")
        
except Exception as e:
    print(f"\n❌ ERRO na configuração do dataset: {e}")
    print("\n🔧 SOLUÇÃO MANUAL:")
    print("1. Verifique se o ZIP contém as imagens corretas")
    print("2. Certifique-se que as imagens estão em formato PNG 1024x1024")
    print("3. Tente novamente com um ZIP válido")

In [ ]:
# 🔍 Verificar imagens enviadas
import os
from PIL import Image

dataset_files = [f for f in os.listdir(DATASET_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
dataset_files.sort()

print(f"📊 Imagens encontradas: {len(dataset_files)}")

if len(dataset_files) == 0:
    print("❌ Nenhuma imagem encontrada! Faça upload das imagens primeiro.")
else:
    print("\n📸 Lista de imagens:")
    for i, filename in enumerate(dataset_files, 1):
        file_path = os.path.join(DATASET_PATH, filename)
        try:
            with Image.open(file_path) as img:
                print(f"{i:2d}. {filename} - {img.size[0]}x{img.size[1]} - {img.format}")
        except Exception as e:
            print(f"{i:2d}. {filename} - ERRO: {e}")
    
    # Verificar se temos as 18 imagens esperadas
    if len(dataset_files) != DATASET_SIZE:
        print(f"\n⚠️ Esperado: {DATASET_SIZE} imagens, Encontrado: {len(dataset_files)}")
        print("   Adicione mais imagens se necessário.")
    else:
        print(f"\n✅ Dataset completo: {DATASET_SIZE} imagens!")

In [ ]:
# 📝 Gerar legendas SIMPLES para LoRA de identidade facial
import os
import json

def create_captions_file():
    """
    Cria legendas SIMPLES para LoRA de identidade facial.
    
    ⚠️ IMPORTANTE: Para LoRA de identidade, legendas SIMPLES são melhores que detalhadas!
    
    ✅ VANTAGENS das legendas simples:
    • Foco na identidade facial (não em contextos)
    • Evita overfitting em roupas/cenários específicos
    • Melhor generalização para novos prompts
    • LoRA mais versátil e flexível
    """
    captions = {}
    
    # Lista de variações SIMPLES de legenda (ideal para identidade)
    caption_variations = [
        f"a photo of {TRIGGER_WORD}",
        f"portrait of {TRIGGER_WORD}",
        f"{TRIGGER_WORD} smiling",
        f"{TRIGGER_WORD} looking at camera",
        f"close-up photo of {TRIGGER_WORD}",
        f"{TRIGGER_WORD} portrait photo",
        f"high quality photo of {TRIGGER_WORD}",
        f"{TRIGGER_WORD} face portrait",
    ]
    
    dataset_files = [f for f in os.listdir(DATASET_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    dataset_files.sort()
    
    for i, filename in enumerate(dataset_files):
        # Usar variação de legenda baseada no índice
        caption = caption_variations[i % len(caption_variations)]
        captions[filename] = caption
    
    # Salvar legendas em arquivo JSON
    captions_file = os.path.join(DATASET_PATH, "captions.json")
    with open(captions_file, 'w') as f:
        json.dump(captions, f, indent=2)
    
    return captions, captions_file

# Gerar legendas
captions, captions_file = create_captions_file()

print("📝 Legendas SIMPLES geradas (ideais para LoRA de identidade):")
print(f"📁 Arquivo: {captions_file}")
print("\n💡 POR QUE LEGENDAS SIMPLES?")
print("   • Foco na identidade facial (não em contextos)")
print("   • Evita overfitting em roupas/cenários")
print("   • Melhor generalização para novos prompts")
print("   • LoRA mais versátil e flexível")
print("\n📋 Legendas por imagem:")
for filename, caption in captions.items():
    print(f"   {filename}: '{caption}'")

print(f"\n✅ {len(captions)} legendas criadas com trigger word '{TRIGGER_WORD}'!")
print("\n⚠️ EVITAMOS descrições detalhadas como:")
print("   ❌ 'vltna woman, 25 years old, brown hair, blue eyes, red dress...'")
print("   ✅ Melhor: 'portrait of vltna woman' (simples e efetivo)")

## 🎛️ Etapa 4: Inicialização do FluxGym

Execução da interface FluxGym com sistema robusto de detecção de URLs e tratamento de erros.

### 🔧 **Sistema de Inicialização Melhorado:**
- **Detecção automática de URLs**: Múltiplos padrões para capturar URLs públicas
- **Tratamento de erros**: Correção automática de problemas de compatibilidade  
- **Múltiplas tentativas**: Até 3 tentativas com diferentes abordagens
- **Diagnósticos integrados**: Verificação de status e solução de problemas

### 📋 **Células Disponíveis:**
1. **Inicialização Principal**: Startup automático com detecção de URL
2. **Verificação de Status**: Para checar se FluxGym está rodando
3. **Inicialização Manual**: Backup caso a automática falhe
4. **Diagnósticos**: Solução de problemas e verificação do sistema

### 🎯 **Execute em Ordem:**
1. Primeira execute a **Inicialização Principal** abaixo
2. Se não aparecer URL, execute **Verificação de Status**
3. Como último recurso, use **Inicialização Manual**
4. Use **Diagnósticos** se encontrar problemas

⚠️ **IMPORTANTE**: Aguarde cada célula terminar antes de executar a próxima!

In [ ]:
# 🚀 INICIALIZAÇÃO FluxGym com URLs Públicas FORÇADAS
# Esta célula resolve o problema de URLs locais não acessíveis

%cd /content/fluxgym-Colab/

import subprocess
import threading
import time
import re
import os
import signal
from IPython.display import display, HTML, clear_output

print("🚀 INICIALIZAÇÃO COMPLETA DO FLUXGYM")
print("="*60)
print("🎯 Objetivo: Criar URL pública acessível via browser")
print("⚠️ Esta célula DEVE permanecer ativa durante o treinamento!")
print("\n🧹 Limpando processos anteriores...")

# Matar todos os processos FluxGym/Gradio anteriores
!pkill -f "python.*app.py" 2>/dev/null || true
!pkill -f "gradio" 2>/dev/null || true
!pkill -f "ngrok" 2>/dev/null || true
time.sleep(3)

# Variáveis globais para controle
fluxgym_process = None
ngrok_process = None
monitoring_active = True
public_url_found = None

def install_ngrok_if_needed():
    """Instala ngrok se necessário"""
    try:
        result = subprocess.run(["which", "ngrok"], capture_output=True)
        if result.returncode != 0:
            print("📦 Instalando ngrok...")
            !curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
            !echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
            !sudo apt update && sudo apt install ngrok -y
            print("✅ ngrok instalado!")
        else:
            print("✅ ngrok já disponível!")
        return True
    except Exception as e:
        print(f"❌ Erro instalando ngrok: {e}")
        return False

def start_fluxgym_local():
    """Inicia FluxGym em modo local na porta 7860"""
    global fluxgym_process
    
    print("🔄 Iniciando FluxGym em modo local...")
    
    try:
        fluxgym_process = subprocess.Popen(
            ["python", "app.py", "--server-port", "7860", "--server-name", "0.0.0.0"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            cwd="/content/fluxgym-Colab",
            env=dict(os.environ, PYTHONUNBUFFERED="1")
        )
        
        print("⏳ Aguardando FluxGym inicializar...")
        time.sleep(15)  # Dar tempo para FluxGym inicializar
        
        # Verificar se o processo está rodando
        if fluxgym_process.poll() is None:
            print("✅ FluxGym iniciado com sucesso na porta 7860!")
            return True
        else:
            print("❌ FluxGym falhou ao inicializar")
            return False
            
    except Exception as e:
        print(f"❌ Erro iniciando FluxGym: {e}")
        return False

def create_ngrok_tunnel():
    """Cria túnel ngrok para porta 7860"""
    global ngrok_process, public_url_found
    
    print("🌐 Criando túnel ngrok para porta 7860...")
    
    try:
        ngrok_process = subprocess.Popen(
            ["ngrok", "http", "7860", "--log=stdout"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        
        print("🔍 Procurando URL do ngrok...")
        start_time = time.time()
        
        while time.time() - start_time < 45:  # 45 segundos para encontrar URL
            line = ngrok_process.stdout.readline()
            if line:
                line = line.strip()
                # Procurar por URLs ngrok
                url_match = re.search(r'url=(https://[\w\d.-]+\.ngrok\.io)', line)
                if not url_match:
                    url_match = re.search(r'(https://[\w\d.-]+\.ngrok\.io)', line)
                
                if url_match:
                    public_url_found = url_match.group(1)
                    print(f"🎉 TÚNEL NGROK CRIADO: {public_url_found}")
                    
                    # Exibir link clicável imediatamente
                    display(HTML(f'''
                    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 25px; border-radius: 15px; margin: 20px 0; color: white; box-shadow: 0 10px 30px rgba(0,0,0,0.3);">
                        <h2 style="margin: 0 0 15px 0; font-size: 24px;">🌐 FluxGym Pronto via Ngrok!</h2>
                        <p style="margin: 10px 0; font-size: 18px; opacity: 0.9;">URL Pública Acessível:</p>
                        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px; margin: 15px 0; font-family: monospace; font-size: 16px; word-break: break-all;">
                            {public_url_found}
                        </div>
                        <a href="{public_url_found}" target="_blank" style="
                            background: #4CAF50; 
                            color: white; 
                            padding: 15px 40px; 
                            text-decoration: none; 
                            border-radius: 25px; 
                            font-weight: bold;
                            font-size: 18px;
                            display: inline-block;
                            margin-top: 15px;
                            box-shadow: 0 5px 15px rgba(76,175,80,0.3);
                            transition: all 0.3s ease;
                        " onmouseover="this.style.transform='translateY(-2px)'" onmouseout="this.style.transform='translateY(0)'">
                            🚀 ABRIR FLUXGYM AGORA
                        </a>
                    </div>
                    '''))
                    
                    return public_url_found
            
            time.sleep(1)
        
        print("⏰ Timeout aguardando URL do ngrok")
        return None
        
    except Exception as e:
        print(f"❌ Erro criando túnel ngrok: {e}")
        return None

def try_gradio_share():
    """Tenta iniciar FluxGym com compartilhamento Gradio nativo"""
    global fluxgym_process, public_url_found
    
    print("🔄 Tentativa 1: Gradio Share nativo...")
    
    try:
        fluxgym_process = subprocess.Popen(
            ["python", "app.py", "--share", "--server-port", "7860"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            cwd="/content/fluxgym-Colab",
            env=dict(os.environ, PYTHONUNBUFFERED="1")
        )
        
        print("🔍 Aguardando URL pública do Gradio...")
        start_time = time.time()
        
        while time.time() - start_time < 90:  # 90 segundos para Gradio share
            line = fluxgym_process.stdout.readline()
            if line:
                line = line.strip()
                print(f"📤 {line}")
                
                # Procurar URLs públicas do Gradio
                public_patterns = [
                    r'Running on public URL: (https://[\w\d.-]+\.gradio\.live)',
                    r'Public URL: (https://[\w\d.-]+\.gradio\.live)',
                    r'(https://[\w\d.-]+\.gradio\.live)'
                ]
                
                for pattern in public_patterns:
                    match = re.search(pattern, line)
                    if match:
                        public_url_found = match.group(1)
                        print(f"🎉 URL PÚBLICA GRADIO: {public_url_found}")
                        
                        display(HTML(f'''
                        <div style="background: linear-gradient(135deg, #ff6b6b 0%, #ee5a24 100%); padding: 25px; border-radius: 15px; margin: 20px 0; color: white; box-shadow: 0 10px 30px rgba(0,0,0,0.3);">
                            <h2 style="margin: 0 0 15px 0; font-size: 24px;">🌟 FluxGym Pronto via Gradio!</h2>
                            <p style="margin: 10px 0; font-size: 18px; opacity: 0.9;">URL Pública:</p>
                            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px; margin: 15px 0; font-family: monospace; font-size: 16px; word-break: break-all;">
                                {public_url_found}
                            </div>
                            <a href="{public_url_found}" target="_blank" style="
                                background: #2196F3; 
                                color: white; 
                                padding: 15px 40px; 
                                text-decoration: none; 
                                border-radius: 25px; 
                                font-weight: bold;
                                font-size: 18px;
                                display: inline-block;
                                margin-top: 15px;
                                box-shadow: 0 5px 15px rgba(33,150,243,0.3);
                            ">🚀 ABRIR FLUXGYM</a>
                        </div>
                        '''))
                        
                        return public_url_found
            
            if fluxgym_process.poll() is not None:
                print("⚠️ Processo Gradio encerrado prematuramente")
                break
        
        print("⏰ Timeout - Gradio share não funcionou")
        if fluxgym_process:
            fluxgym_process.terminate()
        return None
        
    except Exception as e:
        print(f"❌ Erro com Gradio share: {e}")
        return None

def monitor_and_keep_alive():
    """Monitora e mantém processos ativos"""
    global fluxgym_process, ngrok_process, monitoring_active, public_url_found
    
    print(f"\n✅ FluxGym acessível via: {public_url_found}")
    print("\n📱 PRÓXIMOS PASSOS:")
    print("1. Clique no link acima para acessar a interface")
    print("2. Faça upload das 18 imagens da Valentina")
    print("3. Configure:")
    print("   • LoRA Name: valentina_flux_identity_lora")
    print("   • Trigger Word: vltna woman")
    print("   • Repeat: 10")
    print("   • Epochs: 10")
    print("4. Inicie o treinamento")
    print("\n⚠️ IMPORTANTE: NÃO FECHE esta célula durante o treinamento!")
    print("💓 Monitorando processo...")
    
    try:
        while monitoring_active:
            # Verificar se FluxGym ainda está rodando
            if fluxgym_process and fluxgym_process.poll() is not None:
                print("\n⚠️ FluxGym encerrou! Tentando reiniciar...")
                start_fluxgym_local()
            
            # Verificar se ngrok ainda está rodando
            if ngrok_process and ngrok_process.poll() is not None:
                print("\n⚠️ Ngrok encerrou! Tentando recriar túnel...")
                create_ngrok_tunnel()
            
            # Status heartbeat
            current_time = time.strftime('%H:%M:%S')
            print(f"\r💓 Ativo - {current_time} - URL: {public_url_found}", end="", flush=True)
            
            time.sleep(30)  # Verificar a cada 30 segundos
            
    except KeyboardInterrupt:
        print("\n\n⏹️ Parando FluxGym...")
        monitoring_active = False
        if fluxgym_process:
            fluxgym_process.terminate()
        if ngrok_process:
            ngrok_process.terminate()
        print("✅ Processos encerrados.")

# EXECUÇÃO PRINCIPAL
print("\n🚀 INICIANDO SISTEMA COMPLETO...")

# 1. Instalar ngrok
if not install_ngrok_if_needed():
    print("❌ Falha instalando ngrok. Tentando apenas Gradio share...")

# 2. Tentativa 1: Gradio Share
public_url = try_gradio_share()

# 3. Tentativa 2: Local + Ngrok (se Gradio falhou)
if not public_url:
    print("\n🔄 Método 2: Local + Ngrok...")
    
    if start_fluxgym_local():
        public_url = create_ngrok_tunnel()
    else:
        print("❌ Falha iniciando FluxGym local")

# 4. Resultado final
if public_url:
    public_url_found = public_url
    print(f"\n🎉 SUCESSO! FluxGym acessível via: {public_url_found}")
    
    # Manter processo ativo
    monitor_and_keep_alive()
    
else:
    print("\n❌ FALHA: Não foi possível criar URL pública")
    print("\n🔧 SOLUÇÕES MANUAIS:")
    print("1. Reinicie o runtime do Colab")
    print("2. Execute novamente todas as células de instalação")
    print("3. Verifique sua conexão de internet")
    print("4. Tente a célula de inicialização manual abaixo")

In [ ]:
# 🔍 Verificar Status do FluxGym
# Execute esta célula se a anterior não mostrou URLs ou se perdeu a conexão

import subprocess
import requests
import time
import re
import os
from IPython.display import display, HTML

def check_fluxgym_status():
    """Verifica se FluxGym está rodando e tenta encontrar URLs"""
    
    print("🔍 Verificando status do FluxGym...")
    
    # Verificar processos Python rodando
    try:
        result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
        python_processes = [line for line in result.stdout.split('\n') if 'python' in line and 'app.py' in line]
        
        if python_processes:
            print("✅ FluxGym encontrado em execução:")
            for proc in python_processes:
                print(f"   🔸 {proc.split()[1]}: {' '.join(proc.split()[10:12])}")
        else:
            print("❌ Nenhum processo FluxGym detectado")
            return False
    except:
        print("⚠️ Não foi possível verificar processos")
    
    # Tentar encontrar URLs ativas
    print("\n🔍 Procurando por URLs ativas...")
    
    # Verificar portas comuns do Gradio
    common_ports = [7860, 7861, 7862, 8080, 8000]
    found_urls = []
    
    for port in common_ports:
        try:
            url = f"http://127.0.0.1:{port}"
            response = requests.get(url, timeout=3)
            if response.status_code == 200:
                found_urls.append(url)
                print(f"✅ Interface ativa encontrada: {url}")
        except:
            continue
    
    # Verificar logs recentes do sistema para URLs Gradio
    try:
        result = subprocess.run(["journalctl", "--since", "5 minutes ago"], capture_output=True, text=True)
        gradio_lines = [line for line in result.stdout.split('\n') if 'gradio' in line.lower() and 'http' in line]
        
        url_patterns = [
            r'(https://[\w\d.-]+\.gradio\.live)',
            r'(https://[\w\d.-]+\.ngrok\.io)',
            r'(http://[\d.:]+)'
        ]
        
        for line in gradio_lines:
            for pattern in url_patterns:
                matches = re.findall(pattern, line, re.IGNORECASE)
                for match in matches:
                    if match not in found_urls:
                        found_urls.append(match)
                        print(f"📋 URL encontrada nos logs: {match}")
    except:
        print("⚠️ Não foi possível verificar logs do sistema")
    
    return found_urls

# Executar verificação
print("🔍 VERIFICAÇÃO DE STATUS DO FLUXGYM")
print("="*50)

active_urls = check_fluxgym_status()

if active_urls:
    print("\n🎉 URLs ATIVAS ENCONTRADAS:")
    for i, url in enumerate(active_urls, 1):
        print(f"\n🔗 URL {i}: {url}")
        # Criar link clicável
        display(HTML(f'<h3><a href="{url}" target="_blank" style="color: #4CAF50;">🚀 ACESSAR FLUXGYM - URL {i}</a></h3>'))
    
    print("\n📱 INSTRUÇÕES:")
    print("1. Clique em um dos links acima")
    print("2. Se não carregar, tente a próxima URL")
    print("3. Configure seu treinamento LoRA")
    
else:
    print("\n❌ Nenhuma URL ativa encontrada.")
    print("\n🔧 SOLUÇÕES:")
    print("1. Execute novamente a célula anterior de inicialização")
    print("2. Aguarde mais tempo para FluxGym inicializar")
    print("3. Reinicie o runtime e execute tudo novamente")
    print("4. Verifique se há espaço suficiente em disco")
    
    print("\n💡 ALTERNATIVA MANUAL:")
    print("Execute o comando abaixo em uma nova célula para inicializar manualmente:")
    print("```")
    print("%cd /content/fluxgym-Colab/")
    print("!python app.py --share")
    print("```")

print("\n⏰ Esta verificação pode ser executada quantas vezes necessário.")

In [ ]:
# 🛠️ INICIALIZAÇÃO MANUAL DO FLUXGYM (Backup)
# Execute esta célula APENAS se as células anteriores falharam

print("🛠️ INICIALIZAÇÃO MANUAL DO FLUXGYM")
print("="*50)
print("⚠️ Execute esta célula apenas se as anteriores falharam!")
print("\n🔄 Iniciando FluxGym manualmente...")

%cd /content/fluxgym-Colab/

# Limpar processos anteriores
print("🧹 Limpando processos anteriores...")
!pkill -f "python.*app.py" 2>/dev/null || true
!pkill -f "gradio" 2>/dev/null || true

# Aguardar limpeza
import time
time.sleep(3)

print("\n🚀 Iniciando FluxGym com forçar compartilhamento público...")
print("⏳ Esta operação pode demorar alguns minutos...")
print("🌐 Forçando criação de URL pública...")
print("\n📺 AGUARDE A URL APARECER ABAIXO:")
print("-" * 40)

# Tentar múltiplas abordagens para garantir URL pública
import subprocess
import threading
import time
import re
from IPython.display import display, HTML

# Método 1: Iniciar com argumentos de compartilhamento máximo
print("🔄 Método 1: Tentando com argumentos de compartilhamento...")
try:
    process = subprocess.Popen(
        ["python", "app.py", "--share", "--server-name", "0.0.0.0", "--server-port", "7860", "--enable-queue"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    # Monitorar por 60 segundos procurando URL pública
    start_time = time.time()
    public_url_found = False
    
    while time.time() - start_time < 60 and not public_url_found:
        line = process.stdout.readline()
        if line:
            line = line.strip()
            print(f"📤 {line}")
            
            # Procurar por URL pública
            public_patterns = [
                r'(https://[\w\d.-]+\.gradio\.live)',
                r'Running on public URL: (https://[\w\d.-]+\.gradio\.live)',
                r'Public URL: (https://[\w\d.-]+\.gradio\.live)'
            ]
            
            for pattern in public_patterns:
                match = re.search(pattern, line)
                if match:
                    public_url = match.group(1)
                    print(f"\n🎉 URL PÚBLICA ENCONTRADA: {public_url}")
                    display(HTML(f'''
                    <div style="background: #d4edda; padding: 20px; border-radius: 10px; margin: 15px 0; border: 2px solid #28a745;">
                        <h2 style="color: #155724; margin: 0;">✅ FluxGym Pronto!</h2>
                        <p style="margin: 15px 0; font-size: 16px;"><strong>URL Pública:</strong></p>
                        <p style="margin: 10px 0; font-family: monospace; background: #f8f9fa; padding: 10px; border-radius: 5px;">{public_url}</p>
                        <a href="{public_url}" target="_blank" style="
                            background: #28a745; 
                            color: white; 
                            padding: 15px 30px; 
                            text-decoration: none; 
                            border-radius: 8px; 
                            font-weight: bold;
                            font-size: 16px;
                            display: inline-block;
                            margin-top: 15px;
                            box-shadow: 0 5px 15px rgba(76,175,80,0.3);
                            transition: all 0.3s ease;
                        " onmouseover="this.style.transform='translateY(-2px)'" onmouseout="this.style.transform='translateY(0)'">
                            🚀 ABRIR FLUXGYM AGORA
                        </a>
                    </div>
                    '''))
                    public_url_found = True
                    break
        
        if process.poll() is not None:
            break
    
    if not public_url_found:
        print("\n⚠️ URL pública não detectada no método 1. Tentando método 2...")
        process.terminate()
        time.sleep(2)
        
        # Método 2: Forçar ngrok
        print("🔄 Método 2: Configurando túnel ngrok...")
        
        # Instalar ngrok se necessário
        try:
            !which ngrok || (curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok -y)
            
            # Iniciar FluxGym em segundo plano na porta 7860
            print("🚀 Iniciando FluxGym localmente...")
            local_process = subprocess.Popen(
                ["python", "app.py", "--server-port", "7860"],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            
            # Aguardar FluxGym inicializar
            time.sleep(10)
            
            # Criar túnel ngrok
            print("🌐 Criando túnel ngrok...")
            ngrok_process = subprocess.Popen(
                ["ngrok", "http", "7860", "--log=stdout"],
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True
            )
            
            # Procurar URL do ngrok
            ngrok_start = time.time()
            while time.time() - ngrok_start < 30:
                line = ngrok_process.stdout.readline()
                if line and "https://" in line and "ngrok.io" in line:
                    url_match = re.search(r'(https://[\w\d.-]+\.ngrok\.io)', line)
                    if url_match:
                        ngrok_url = url_match.group(1)
                        print(f"\n🎉 TÚNEL NGROK CRIADO: {ngrok_url}")
                        display(HTML(f'''
                        <div style="background: #fff3cd; padding: 20px; border-radius: 10px; margin: 15px 0; border: 2px solid #ffc107;">
                            <h2 style="color: #856404; margin: 0;">🔗 FluxGym via Ngrok</h2>
                            <p style="margin: 15px 0; font-size: 16px;"><strong>URL Ngrok:</strong></p>
                            <p style="margin: 10px 0; font-family: monospace; background: #f8f9fa; padding: 10px; border-radius: 5px;">{ngrok_url}</p>
                            <a href="{ngrok_url}" target="_blank" style="
                                background: #ffc107; 
                                color: #212529; 
                                padding: 15px 30px; 
                                text-decoration: none; 
                                border-radius: 8px; 
                                font-weight: bold;
                                font-size: 16px;
                                display: inline-block;
                                margin-top: 15px;
                            ">🚀 ABRIR VIA NGROK</a>
                        </div>
                        '''))
                        public_url_found = True
                        break
                time.sleep(1)
            
        except Exception as e:
            print(f"❌ Erro configurando ngrok: {e}")
    
    if not public_url_found:
        print("\n⚠️ Não foi possível criar URL pública automaticamente.")
        print("\n🔧 SOLUÇÕES MANUAIS:")
        print("1. Aguarde mais alguns minutos - às vezes demora")
        print("2. Tente acessar: http://localhost:7860 (se disponível)")
        print("3. Reinicie o runtime e tente novamente")
        print("4. Verifique os logs acima para URLs que possam ter aparecido")
        
        # Mostrar processo ativo
        if process.poll() is None:
            print("\n💓 FluxGym está rodando em segundo plano.")
            print("⚠️ Não feche esta célula!")
        else:
            print("\n❌ Processo FluxGym encerrado.")
    
    else:
        print("\n✅ FluxGym configurado com sucesso!")
        print("\n📱 PRÓXIMOS PASSOS:")
        print("1. Clique no link acima para acessar FluxGym")
        print("2. Faça upload do dataset (18 imagens)")
        print("3. Configure:")
        print("   • LoRA Name: valentina_flux_identity_lora")
        print("   • Trigger Word: vltna woman")
        print("   • Repeat: 10")
        print("   • Epochs: 10")
        print("4. Inicie o treinamento")
        print("\n💓 Mantenha esta célula ativa durante o treinamento!")

except Exception as e:
    print(f"\n❌ Erro na inicialização manual: {e}")
    print("\n🔧 TENTE:")
    print("1. Reinicie o runtime")
    print("2. Execute as células de instalação novamente")
    print("3. Verifique se há espaço em disco suficiente")

print("\n⚠️ IMPORTANTE: Mantenha esta célula rodando durante todo o treinamento!")

In [ ]:
# 🔧 DIAGNÓSTICOS E SOLUÇÃO DE PROBLEMAS
# Execute para diagnosticar problemas com FluxGym

print("🔧 DIAGNÓSTICOS DO SISTEMA")
print("="*50)

# Verificar espaço em disco
print("\n💾 ESPAÇO EM DISCO:")
!df -h /content

# Verificar memória
print("\n🧠 MEMÓRIA:")
!free -h

# Verificar GPU
print("\n🎮 GPU:")
!nvidia-smi --query-gpu=name,memory.total,memory.used --format=csv

# Verificar instalação do FluxGym
print("\n📦 VERIFICAÇÃO DO FLUXGYM:")
import os

fluxgym_path = "/content/fluxgym-Colab"
if os.path.exists(fluxgym_path):
    print(f"✅ FluxGym encontrado em: {fluxgym_path}")
    
    # Verificar arquivos principais
    key_files = ['app.py', 'requirements.txt', 'models/', 'sd-scripts/']
    for file in key_files:
        file_path = os.path.join(fluxgym_path, file)
        if os.path.exists(file_path):
            print(f"  ✅ {file} - OK")
        else:
            print(f"  ❌ {file} - AUSENTE")
else:
    print(f"❌ FluxGym não encontrado em: {fluxgym_path}")

# Verificar modelos
print("\n🤖 MODELOS:")
models_path = "/content/fluxgym-Colab/models"
if os.path.exists(models_path):
    for subdir in ['unet', 'clip', 'vae']:
        subdir_path = os.path.join(models_path, subdir)
        if os.path.exists(subdir_path):
            files = os.listdir(subdir_path)
            if files:
                print(f"  ✅ {subdir}: {len(files)} arquivo(s)")
                for file in files:
                    size = os.path.getsize(os.path.join(subdir_path, file)) / (1024**3)
                    print(f"    📄 {file} ({size:.1f} GB)")
            else:
                print(f"  ⚠️ {subdir}: vazio")
        else:
            print(f"  ❌ {subdir}: não encontrado")
else:
    print("❌ Pasta de modelos não encontrada")

# Verificar dataset
print("\n📊 DATASET:")
dataset_path = "/content/valentina_dataset"
if os.path.exists(dataset_path):
    images = [f for f in os.listdir(dataset_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"✅ Dataset encontrado: {len(images)} imagens")
    if len(images) < 18:
        print(f"⚠️ Recomendado: 18+ imagens (atual: {len(images)})")
else:
    print("⚠️ Dataset não encontrado - execute a célula de upload primeiro")

# Verificar processos Python
print("\n🔄 PROCESSOS PYTHON ATIVOS:")
!ps aux | grep python | grep -v grep

# Verificar portas em uso
print("\n🔌 PORTAS EM USO:")
!netstat -tlnp 2>/dev/null | grep :78 || echo "Nenhuma porta 78xx detectada"

# Verificar logs recentes
print("\n📋 LOGS RECENTES (últimas 10 linhas):")
try:
    with open('/content/fluxgym-Colab/app.log', 'r') as f:
        lines = f.readlines()
        for line in lines[-10:]:
            print(f"  {line.strip()}")
except:
    print("  ⚠️ Nenhum log encontrado")

print("\n" + "="*50)
print("✅ Diagnósticos concluídos!")
print("\n💡 Se encontrar problemas:")
print("1. Verifique se há espaço suficiente em disco (>10 GB livres)")
print("2. Confirme que todos os modelos foram baixados")
print("3. Certifique-se de que o dataset foi enviado")
print("4. Reinicie o runtime se necessário")

## ⚙️ Etapa 5: Configuração Manual do Treinamento

**ATENÇÃO**: Execute a configuração manual na interface web do FluxGym que foi aberta acima.

### 🎯 Configurações Recomendadas na Interface FluxGym:

#### **LoRA Info:**
- **LoRA Name**: `valentina_flux_identity_lora`
- **Trigger Word**: `vltna woman`
- **VRAM Setting**: 20GB (ou maior disponível)
- **Repeat Count**: 10
- **Max Train Epochs**: 10
- **Learning Rate**: 1e-4
- **Network Rank**: 16
- **Batch Size**: 2 (se possível)

#### **Dataset:**
1. Faça upload das 18 imagens através da interface
2. Preencha as legendas com variações de `vltna woman`
3. Verifique se todas as imagens têm legendas

#### **Training:**
1. Revise as configurações finais
2. Clique em "Start Training"
3. **NÃO FECHE esta aba do Colab** durante o treinamento

### ⏱️ Tempo Estimado: 2-3 horas
### 🔥 **IMPORTANTE**: Monitore o progresso e mantenha o Colab ativo!

In [ ]:
# 📊 Monitor de progresso do treinamento
import os
import time
import glob
from datetime import datetime

def monitor_training():
    """Monitora o progresso do treinamento verificando logs e outputs"""
    
    print("📊 Monitor de Treinamento Iniciado")
    print("=" * 50)
    print(f"⏰ Início: {datetime.now().strftime('%H:%M:%S')}")
    print("🔍 Monitorando pasta de outputs e logs...")
    print("\n📋 Para parar o monitoramento: Interromper célula (Ctrl+C)")
    
    last_file_count = 0
    start_time = time.time()
    
    try:
        while True:
            # Verificar arquivos na pasta de output
            output_files = glob.glob(f"{OUTPUT_PATH}/**/*", recursive=True)
            current_file_count = len(output_files)
            
            # Verificar se há novos arquivos
            if current_file_count != last_file_count:
                print(f"\n📁 Arquivos detectados: {current_file_count}")
                
                # Listar arquivos .safetensors (LoRA outputs)
                lora_files = [f for f in output_files if f.endswith('.safetensors')]
                if lora_files:
                    print("🎯 LoRAs encontrados:")
                    for lora_file in lora_files:
                        size_mb = os.path.getsize(lora_file) / (1024*1024)
                        print(f"   📄 {os.path.basename(lora_file)} ({size_mb:.1f} MB)")
                
                last_file_count = current_file_count
            
            # Mostrar tempo decorrido
            elapsed = time.time() - start_time
            hours, remainder = divmod(elapsed, 3600)
            minutes, seconds = divmod(remainder, 60)
            print(f"\r⏰ Tempo decorrido: {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}", end="")
            
            time.sleep(30)  # Verificar a cada 30 segundos
            
    except KeyboardInterrupt:
        print(f"\n\n⏹️ Monitoramento interrompido pelo usuário")
        print(f"⏰ Tempo total: {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}")

# Iniciar monitoramento
monitor_training()

## 📥 Etapa 6: Download do LoRA Treinado

Após o treinamento concluir, faça download do arquivo LoRA gerado.

In [ ]:
# 🔍 Localizar e baixar o LoRA treinado
import os
import glob
from google.colab import files

print("🔍 Procurando arquivos LoRA gerados...")

# Procurar por arquivos .safetensors na pasta de outputs
lora_files = glob.glob(f"{OUTPUT_PATH}/**/*.safetensors", recursive=True)

if not lora_files:
    print("❌ Nenhum arquivo LoRA encontrado!")
    print("   Verifique se o treinamento foi concluído com sucesso.")
    print(f"   Pasta verificada: {OUTPUT_PATH}")
else:
    print(f"✅ {len(lora_files)} arquivo(s) LoRA encontrado(s):")
    
    for lora_file in lora_files:
        file_size = os.path.getsize(lora_file) / (1024*1024)  # MB
        file_name = os.path.basename(lora_file)
        print(f"\n📄 {file_name}")
        print(f"   📏 Tamanho: {file_size:.1f} MB")
        print(f"   📁 Caminho: {lora_file}")
        
        # Fazer download do arquivo
        print(f"📥 Iniciando download de {file_name}...")
        try:
            files.download(lora_file)
            print(f"✅ Download concluído: {file_name}")
        except Exception as e:
            print(f"❌ Erro no download: {e}")

print("\n🎉 Processo de download concluído!")
print("\n📋 Próximos passos:")
print("1. Teste o LoRA em ComfyUI ou Automatic1111")
print("2. Use o trigger word 'vltna woman' nos seus prompts")
print("3. Ajuste o peso do LoRA entre 0.8-1.2 conforme necessário")

## 🧪 Etapa 7: Configuração para Teste do LoRA

Instruções para testar o LoRA treinado em diferentes interfaces.

In [ ]:
# 📋 Gerar guia de uso do LoRA
lora_info = f"""
🎯 VALENTINA FLUX LORA - GUIA DE USO
{'='*50}

📄 Arquivo: {LORA_NAME}.safetensors
🎨 Trigger Word: "{TRIGGER_WORD}"
🎛️ Modelo Base: FLUX.1-dev
📊 Parâmetros: Rank {NETWORK_RANK}, {DATASET_SIZE} imagens, {DATASET_SIZE * REPEAT_COUNT * MAX_EPOCHS} steps

🔧 CONFIGURAÇÕES RECOMENDADAS:
├── LoRA Weight: 0.8 - 1.0 (comece com 0.8)
├── Steps: 25 - 35 (25 ideal para velocidade)
├── CFG Scale: 3.5 - 4.0 (diferente do SD, use valores baixos)
├── Sampler: Euler, DPM++ 2M
├── Resolução: 1024x1024 (resolução de treino)
└── Scheduler: Normal (não use Karras)

✨ PROMPTS DE EXEMPLO:
1. "portrait photo of {TRIGGER_WORD}, professional lighting, high detail"
2. "{TRIGGER_WORD} smiling, outdoor setting, golden hour"
3. "close-up photo of {TRIGGER_WORD}, studio lighting, sharp focus"
4. "{TRIGGER_WORD} wearing elegant dress, photorealistic"

⚠️ DICAS IMPORTANTES:
• SEMPRE inclua "{TRIGGER_WORD}" no prompt
• Flux prefere prompts descritivos e longos
• Não use negative prompts (Flux não precisa)
• CFG Scale alto (>5) pode degradar a qualidade
• Teste diferentes pesos do LoRA se não estiver satisfeito

🛠️ COMPATIBILIDADE:
✅ ComfyUI (recomendado)
✅ Automatic1111 (com extensões FLUX)
❌ diffusers/PyTorch (não suportado nativamente)

📁 INSTALAÇÃO:
1. Coloque o arquivo .safetensors na pasta loras/ da sua interface
2. Reinicie a interface se necessário
3. Selecione o LoRA na interface
4. Configure o peso (0.8-1.0)
5. Use o trigger word nos prompts

🎉 Divirta-se criando imagens da Valentina!
"""

print(lora_info)

# Salvar guia em arquivo
guide_file = f"/content/{LORA_NAME}_usage_guide.txt"
with open(guide_file, 'w', encoding='utf-8') as f:
    f.write(lora_info)

print(f"\n📁 Guia salvo em: {guide_file}")

# Download do guia
try:
    files.download(guide_file)
    print("📥 Guia de uso baixado!")
except:
    print("ℹ️ Guia disponível no Colab para consulta")

## 🎉 Conclusão

### ✅ Treinamento Completo!

Se chegou até aqui, você deve ter:

1. **✅ LoRA treinado** da Valentina com identidade facial
2. **✅ Arquivo .safetensors** baixado (~20-40MB)
3. **✅ Guia de uso** com configurações recomendadas

### 🚀 Próximos Passos:

1. **Teste em ComfyUI**: 
   - Instale ComfyUI localmente
   - Coloque o LoRA na pasta `models/loras/`
   - Use workflow para FLUX.1-dev + LoRA

2. **Prompts Recomendados**:
   ```
   portrait photo of vltna woman, professional lighting, high detail, 1024x1024
   vltna woman smiling, outdoor setting, golden hour lighting
   close-up photo of vltna woman, studio lighting, sharp focus
   ```

3. **Configurações**:
   - LoRA Weight: 0.8-1.0
   - CFG Scale: 3.5-4.0
   - Steps: 25-35
   - Sampler: Euler/DPM++ 2M

### ⚠️ Solução de Problemas:

- **Rosto não parece a Valentina**: Aumente peso do LoRA (1.0-1.2)
- **Imagem com artefatos**: Diminua peso do LoRA (0.7-0.9)
- **Cores estranhas**: Verifique se está usando o VAE correto do FLUX
- **Não gera a pessoa**: Verifique se incluiu o trigger word

### 🎯 Qualidade Esperada:

Com 18 imagens bem diversificadas e 1800 steps de treinamento, o LoRA deve:
- ✅ Reproduzir características faciais da Valentina consistentemente
- ✅ Funcionar com diferentes roupas, cenários e poses
- ✅ Manter qualidade fotorrealística do FLUX.1-dev
- ✅ Responder bem a prompts descritivos

**🏆 Parabéns! Você criou um LoRA personalizado de alta qualidade!**

In [ ]:
# 🔍 VERIFICAÇÃO AVANÇADA DE STATUS FluxGym
# Execute se a célula anterior não mostrou URLs ou perdeu conexão

import subprocess
import requests
import time
import re
import os
from IPython.display import display, HTML

def comprehensive_status_check():
    """Verificação completa de status do FluxGym"""
    
    print("🔍 VERIFICAÇÃO COMPLETA DE STATUS")
    print("="*50)
    
    # 1. Verificar processos Python
    print("\n🔄 Verificando processos ativos...")
    try:
        result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
        app_processes = [line for line in result.stdout.split('\n') if 'app.py' in line and 'python' in line]
        ngrok_processes = [line for line in result.stdout.split('\n') if 'ngrok' in line]
        
        if app_processes:
            print("✅ FluxGym detectado:")
            for proc in app_processes:
                parts = proc.split()
                if len(parts) > 10:
                    print(f"   🟢 PID {parts[1]}: {' '.join(parts[10:])}")
        else:
            print("❌ FluxGym não detectado")
        
        if ngrok_processes:
            print("✅ Ngrok detectado:")
            for proc in ngrok_processes:
                parts = proc.split()
                if len(parts) > 10:
                    print(f"   🟢 PID {parts[1]}: {' '.join(parts[10:])}")
        else:
            print("⚠️ Ngrok não detectado")
            
    except Exception as e:
        print(f"❌ Erro verificando processos: {e}")
    
    # 2. Verificar portas ativas
    print("\n🔌 Verificando portas...")
    try:
        result = subprocess.run(["netstat", "-tlnp"], capture_output=True, text=True)
        port_7860 = "7860" in result.stdout
        port_4040 = "4040" in result.stdout  # Ngrok admin
        
        if port_7860:
            print("✅ Porta 7860 (FluxGym) ATIVA")
        else:
            print("❌ Porta 7860 (FluxGym) inativa")
            
        if port_4040:
            print("✅ Porta 4040 (Ngrok admin) ativa")
        else:
            print("⚠️ Porta 4040 (Ngrok admin) inativa")
            
    except Exception as e:
        print(f"❌ Erro verificando portas: {e}")
    
    # 3. Testar conexões HTTP
    print("\n🌐 Testando conexões...")
    working_urls = []
    
    test_urls = [
        "http://127.0.0.1:7860",
        "http://localhost:7860",
        "http://0.0.0.0:7860"
    ]
    
    for url in test_urls:
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                working_urls.append(url)
                print(f"✅ {url} - RESPONDENDO")
            else:
                print(f"⚠️ {url} - Status {response.status_code}")
        except requests.exceptions.RequestException:
            print(f"❌ {url} - INACESSÍVEL")
    
    # 4. Procurar URLs públicas em logs
    print("\n📋 Procurando URLs públicas...")
    public_urls = []
    
    # Verificar logs do ngrok
    try:
        if os.path.exists("/tmp/ngrok.log"):
            with open("/tmp/ngrok.log", "r") as f:
                content = f.read()
                ngrok_matches = re.findall(r'(https://[\w\d.-]+\.ngrok\.io)', content)
                public_urls.extend(ngrok_matches)
    except:
        pass
    
    # Verificar via API do ngrok
    try:
        response = requests.get("http://127.0.0.1:4040/api/tunnels", timeout=3)
        if response.status_code == 200:
            data = response.json()
            for tunnel in data.get('tunnels', []):
                public_url = tunnel.get('public_url')
                if public_url and public_url.startswith('https://'):
                    public_urls.append(public_url)
                    print(f"🌐 Ngrok API: {public_url}")
    except:
        print("⚠️ Não foi possível consultar API do ngrok")
    
    # Remover duplicatas
    public_urls = list(set(public_urls))
    
    return working_urls, public_urls

def create_emergency_ngrok():
    """Cria túnel ngrok de emergência"""
    print("\n🚨 CRIANDO TÚNEL NGROK DE EMERGÊNCIA...")
    
    try:
        # Verificar se porta 7860 está ativa
        response = requests.get("http://127.0.0.1:7860", timeout=3)
        if response.status_code != 200:
            print("❌ FluxGym não está rodando na porta 7860")
            return None
            
        # Matar ngrok existente
        !pkill -f "ngrok" 2>/dev/null || true
        time.sleep(2)
        
        # Criar novo túnel
        ngrok_process = subprocess.Popen(
            ["ngrok", "http", "7860", "--log=stdout"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        
        # Aguardar URL
        start_time = time.time()
        while time.time() - start_time < 30:
            line = ngrok_process.stdout.readline()
            if line and "https://" in line and "ngrok.io" in line:
                url_match = re.search(r'(https://[\w\d.-]+\.ngrok\.io)', line)
                if url_match:
                    emergency_url = url_match.group(1)
                    print(f"🎉 TÚNEL DE EMERGÊNCIA: {emergency_url}")
                    return emergency_url
            time.sleep(1)
        
        print("⏰ Timeout criando túnel de emergência")
        return None
        
    except Exception as e:
        print(f"❌ Erro no túnel de emergência: {e}")
        return None

# EXECUÇÃO DA VERIFICAÇÃO
local_urls, public_urls = comprehensive_status_check()

print("\n📊 RESUMO DE STATUS:")
print("="*30)

if public_urls:
    print(f"🎉 {len(public_urls)} URL(s) PÚBLICA(s) ENCONTRADA(s):")
    for i, url in enumerate(public_urls, 1):
        print(f"\n🌐 URL {i}: {url}")
        
        # Testar se está acessível
        try:
            response = requests.get(url, timeout=10)
            status = "✅ FUNCIONANDO" if response.status_code == 200 else f"⚠️ Status {response.status_code}"
        except:
            status = "❌ INACESSÍVEL"
        
        print(f"   Status: {status}")
        
        # Criar link clicável
        color = "#4CAF50" if "FUNCIONANDO" in status else "#ff9800"
        display(HTML(f'''
        <div style="background: linear-gradient(135deg, {color} 0%, {color}dd 100%); padding: 15px; border-radius: 10px; margin: 10px 0; color: white;">
            <h3 style="margin: 0 0 10px 0;">🚀 FluxGym - URL {i}</h3>
            <p style="margin: 5px 0; font-family: monospace; opacity: 0.9;">{url}</p>
            <p style="margin: 5px 0; font-size: 14px;">{status}</p>
            <a href="{url}" target="_blank" style="
                background: rgba(255,255,255,0.2); 
                color: white; 
                padding: 10px 20px; 
                text-decoration: none; 
                border-radius: 5px; 
                font-weight: bold;
                display: inline-block;
                margin-top: 10px;
            ">🔗 ABRIR INTERFACE</a>
        </div>
        '''))

elif local_urls:
    print(f"🏠 {len(local_urls)} URL(s) LOCAL(is) encontrada(s):")
    for url in local_urls:
        print(f"   🔗 {url}")
    
    print("\n⚠️ URLs locais não são acessíveis no browser do Colab!")
    print("🔧 Tentando criar túnel público...")
    
    emergency_url = create_emergency_ngrok()
    if emergency_url:
        display(HTML(f'''
        <div style="background: #ff9800; padding: 20px; border-radius: 10px; margin: 15px 0; color: white;">
            <h3 style="margin: 0 0 15px 0;">🚨 Túnel de Emergência Criado!</h3>
            <p style="margin: 10px 0; font-family: monospace;">{emergency_url}</p>
            <a href="{emergency_url}" target="_blank" style="
                background: #f44336; 
                color: white; 
                padding: 12px 25px; 
                text-decoration: none; 
                border-radius: 5px; 
                font-weight: bold;
                display: inline-block;
                margin-top: 10px;
            ">🚀 ACESSAR VIA EMERGÊNCIA</a>
        </div>
        '''))
    else:
        print("❌ Falha criando túnel de emergência")

else:
    print("❌ NENHUMA URL ENCONTRADA!")
    print("\n🔧 SOLUÇÕES RECOMENDADAS:")
    print("1. Execute a célula de inicialização principal novamente")
    print("2. Aguarde 2-3 minutos para inicialização completa")
    print("3. Verifique se há espaço suficiente em disco")
    print("4. Reinicie o runtime se necessário")
    
    print("\n💡 DIAGNÓSTICO ADICIONAL:")
    print("Execute a célula de diagnósticos avançados abaixo")

print(f"\n⏰ Verificação concluída: {time.strftime('%H:%M:%S')}")
print("🔄 Esta célula pode ser executada repetidamente conforme necessário")